## MGMTMSA 403 Homework 3
### Submitted by- Himanshu Chandra


In [1]:
# Import gurobi and numpy
from gurobipy import *
import numpy as np
from numpy import genfromtxt
import csv
import pandas as pd
train=pd.read_csv("AirbnbTrain.csv")
test=pd.read_csv("AirbnbTest.csv")
## Get index of 4 tickers



ModuleNotFoundError: No module named 'gurobipy'

In [2]:
x_train=train.drop(['price'],axis=1)
y_train=train['price']
#x_train.insert(0, "Intercept", 1) 
n,m=x_train.shape

x_test=test.drop(['price'],axis=1)
#x_test.insert(0, "Intercept", 1)
y_test=test['price']
n_test,m_test=x_test.shape
x_test=np.matrix(x_test)
y_test=np.array(y_test)


In [3]:
x_train=np.matrix(x_train)
y_train=np.array(y_train)


##  Model 1

In [4]:
model1=Model()
b=model1.addVars(m)
z=model1.addVars(n)
const1={}
const2={}

Academic license - for non-commercial use only


In [5]:
for i in range(n):
    const1[i]=model1.addConstr(z[i]>=y_train[i]-sum(b[j]*x_train[i,j] for j in range(m)))
    const2[i]=model1.addConstr(z[i]>=(sum(b[j]*x_train[i,j] for j in range(m))-y_train[i])   )

In [7]:
model1.setObjective(sum(z[i] for i in range(n))/n,GRB.MINIMIZE)

In [8]:
model1.update()
model1

<gurobi.Model Continuous instance Unnamed: 3400 constrs, 1712 vars, Parameter changes: LogFile=gurobi.log, CSIdleTimeout=1800>

In [9]:
model1.optimize()

Optimize a model with 3400 rows, 1712 columns and 41368 nonzeros
Coefficient statistics:
  Matrix range     [5e-01, 5e+02]
  Objective range  [6e-04, 6e-04]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+01, 2e+03]

Concurrent LP optimizer: dual simplex and barrier
Showing barrier log only...

Presolve time: 0.06s
Presolved: 3400 rows, 1712 columns, 41368 nonzeros

Ordering time: 0.00s

Barrier statistics:
 Dense cols : 12
 AA' NZ     : 3.967e+04
 Factor NZ  : 4.315e+04 (roughly 2 MBytes of memory)
 Factor Ops : 5.491e+05 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   2.12302506e+03  0.00000000e+00  1.36e+03 0.00e+00  2.56e+01     0s
   1   2.05671988e+03 -1.60051438e+00  0.00e+00 5.43e-03  1.75e+00     0s
   2   1.14614521e+03  2.86828649e+00  0.00e+00 7.00e-05  2.34e-01     0s
   3   1.13304580e+02  6.92004294e+00  0.00e+00 3.86e-06  2.09

In [15]:
beta=[]
for i in range(m):
    beta.append(b[i].x)
beta=np.array(beta)
error_sum=0
for i in range(n_test):
    error_sum=error_sum+abs(y_test[i]-np.sum(np.array(x_test[i,:])*beta))

print("The prediction error test set in dollars per night is ", error_sum/n_test)

The prediction error test set in dollars per night is  35.62719512642501


## MODEL 2

In [12]:
model2=Model()
b2=model2.addVars(m)
z2=model2.addVars(n)
const12={}
const22={}
const32={}
const42={}
x = model2.addVars(m, vtype = GRB.BINARY)
for i in range(n):
    const12[i]=model2.addConstr(z2[i]>=y_train[i]-sum(b2[j]*x_train[i,j] for j in range(m)))
    const22[i]=model2.addConstr(z2[i]>=(sum(b2[j]*x_train[i,j] for j in range(m))-y_train[i]))
for i in range(m):
    const32[i]=model2.addConstr(1000000*x[i]>=b2[i] )
    const42[i]=model2.addConstr(-1000000*x[i]<=b2[i])
model2.addConstr(sum(x[i] for i in range(m))<=3)
    
model2.setObjective(sum(z2[i] for i in range(n))/n,GRB.MINIMIZE)    

In [13]:
model2.update()
model2

<gurobi.Model MIP instance Unnamed: 3425 constrs, 1724 vars, Parameter changes: LogFile=gurobi.log, CSIdleTimeout=1800>

In [14]:
model2.update()
model2.optimize()

Optimize a model with 3425 rows, 1724 columns and 41428 nonzeros
Variable types: 1712 continuous, 12 integer (12 binary)
Coefficient statistics:
  Matrix range     [5e-01, 1e+06]
  Objective range  [6e-04, 6e-04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+00, 2e+03]
Found heuristic solution: objective 144.9682353
Presolve removed 838 rows and 413 columns
Presolve time: 0.07s
Presolved: 2587 rows, 1311 columns, 31294 nonzeros
Variable types: 1299 continuous, 12 integer (12 binary)

Root relaxation: objective 3.644463e+01, 1393 iterations, 0.24 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   36.44463    0    3  144.96824   36.44463  74.9%     -    0s
H    0     0                      61.6243748   36.44463  40.9%     -    0s
     0     0   36.44463    0    7   61.62437   36.44463  40.9%     -    0s
     0     0   36.44463    0    8   61.62437   36.44

In [17]:
beta

array([ 0.,  0., 52., 14.,  0., 32.,  0.,  0.,  0.,  0.,  0.,  0.])

In [46]:
beta=[]
for i in range(m):
    beta.append(b2[i].x)
beta=np.array(beta)
error_sum=0
for i in range(n_test):
    error_sum=error_sum+abs(y_test[i]-np.sum(np.array(x_test[i,:])*beta))

print("The prediction error test set in dollars per night is ", error_sum/n_test)

The prediction error test set in dollars per night is  37.73676680972818


In [54]:
betap=np.append(beta,[0])
j=0
for i in test.columns[betap>0]:
    print(i, "coefficient is", beta[beta>0][j])
    j=j+1

Entire home coefficient is 52.0
accommodates coefficient is 14.0
bedrooms coefficient is 32.0


## Model3

In [59]:
model3=Model()
b3=model3.addVars(m)
z3=model3.addVars(n)
const13={}
const23={}
const33={}
const43={}
x = model3.addVars(m, vtype = GRB.BINARY)
for i in range(n):
    const13[i]=model3.addConstr(z3[i]>=y_train[i]-sum(b3[j]*x_train[i,j] for j in range(m)))
    const23[i]=model3.addConstr(z3[i]>=(sum(b3[j]*x_train[i,j] for j in range(m))-y_train[i]))
for i in (0,1,2,3,4,5,6,7,8,9,10,11):
    const33[i]=model3.addConstr(10000*x[i]>=b3[i] )
    const43[i]=model3.addConstr(-10000*x[i]<=b3[i])
model3.addConstr(sum(x[i] for i in range(m))<=3)
model3.addConstr(x[6]==1)
    
model3.setObjective(sum(z3[i] for i in range(n))/n,GRB.MINIMIZE)    

In [60]:
model3.update()
model3.optimize()

Optimize a model with 3426 rows, 1724 columns and 41429 nonzeros
Variable types: 1712 continuous, 12 integer (12 binary)
Coefficient statistics:
  Matrix range     [5e-01, 1e+04]
  Objective range  [6e-04, 6e-04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+03]
Found heuristic solution: objective 144.9682353
Presolve removed 840 rows and 414 columns
Presolve time: 0.06s
Presolved: 2586 rows, 1310 columns, 31291 nonzeros
Variable types: 1299 continuous, 11 integer (11 binary)

Root relaxation: objective 3.644463e+01, 1430 iterations, 0.24 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   36.44463    0    8  144.96824   36.44463  74.9%     -    0s
H    0     0                      48.6742700   36.44463  25.1%     -    0s
     0     0   36.45085    0   10   48.67427   36.45085  25.1%     -    0s
H    0     0                      43.9745441   36.45

In [61]:
beta=[]
for i in range(m):
    beta.append(b3[i].x)
beta=np.array(beta)
error_sum=0
for i in range(n_test):
    error_sum=error_sum+abs(y_test[i]-np.sum(np.array(x_test[i,:])*beta))
print("The prediction error test set in dollars per night is ", error_sum/n_test)

The prediction error test set in dollars per night is  38.59960658082976


In [62]:
betap=np.append(beta,[0])
j=0
for i in test.columns[betap>0]:
    print(i, "coefficient is", beta[beta>0][j])
    j=j+1

Entire home coefficient is 67.875
bedrooms coefficient is 47.375
beds coefficient is 12.125


The accommodates column is replaced by the beds column in the model 3 because accommodates had the least important feature in model2 as it had least cofficient among the three. therefore, the optmization algorithm removes the least important feature with beds feature